In [1]:
from flax.training import train_state, checkpoints

In [6]:
CKPT_DIR = 'ckpts'
restored_state = checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=None)


In [7]:
restored_state

{'step': array(2000, dtype=int32),
 'params': {'Decoder_0': {'DecoderLSTM_0': {'Dense_0': {'bias': array([-0.05299965,  0.0051652 , -0.0519604 , -0.05365251,  0.02840759,
             0.06341346,  0.01598888,  0.0202714 ,  0.02348367,  0.0166784 ,
             0.01843355,  0.00784171,  0.01835183,  0.02942931, -0.05430092],
           dtype=float32),
     'kernel': array([[-0.06747954,  0.05155769, -0.07939984, ..., -0.25849   ,
              0.14551093,  0.02747019],
            [ 0.11526638, -0.12333549,  0.0624422 , ...,  0.0894297 ,
             -0.18192708,  0.06651317],
            [ 0.05436617, -0.0492461 ,  0.0210426 , ..., -0.05929632,
              0.10341956,  0.07105672],
            ...,
            [ 0.13008037,  0.01799717,  0.18587229, ..., -0.17262048,
             -0.16038902,  0.09043714],
            [-0.03584237,  0.00787828,  0.0294341 , ..., -0.02481114,
             -0.0661357 , -0.09544624],
            [ 0.03937491, -0.08691473,  0.01453435, ...,  0.16107474,
